In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
import email
import email.policy
from bs4 import BeautifulSoup
from collections import Counter

In [2]:
ham_files = [name for name in sorted(os.listdir("./Dataset/ham/"))]
spam_files = [name for name in sorted(os.listdir("./Dataset/spam//"))]
print("Length of spam_files : ",len(spam_files))
print("Length of ham_files : ",len(ham_files))

Length of spam_files :  501
Length of ham_files :  2551


In [3]:
def email_content(spam_or_not, f_name):
    direc = "./Dataset/spam" if spam_or_not else "./Dataset/ham"
    with open(os.path.join(direc, f_name), "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)
    
ham_emails = [ email_content(spam_or_not=False, f_name=name) for name in ham_files]
spam_emails = [ email_content(spam_or_not=True, f_name=name) for name in spam_files]

In [4]:
(ham_emails[0].get_content_type())

'text/plain'

In [5]:
def email_struc(email):
    if isinstance(email, str):
        return email
    email_payload= email.get_payload()
    if isinstance(email_payload, list):
        return "multipart({})".format(", ".join([email_struc(sub_email) for sub_email in email_payload]))
    else:
        return email.get_content_type()

def structures_counter(emails):
    structures = Counter()
    for email in emails:
        structure = email_struc(email)
        structures[structure] += 1
    return structures

In [6]:
ham_structure = structures_counter(ham_emails)
spam_structure = structures_counter(spam_emails)
for email in spam_emails:
    if email_struc(email) == 'text/html':
        testEmail = email
        break
def html_to_plain(email):
    try:
        soup = BeautifulSoup(email.get_content(), 'html.parser')
        return soup.text.replace('\n\n','')
    except:
        return "empty"

def email_to_plain(email):
    struct = email_struc(email)
    for part in email.walk():
        partContentType = part.get_content_type()
        if partContentType not in ['text/plain','text/html']:
            continue
        try:
            partContent = part.get_content()
        except: # in case of encoding issues
            partContent = str(part.get_payload())
        if partContentType == 'text/plain':
            return partContent
        else:
            return html_to_plain(part)

In [7]:
orig=[]
ham=[]
spam=[]
import pandas as pd 
import re
count=0
for text in ham_emails:
    text=email_to_plain(text)
    try:
        text = text.replace('.','')
        text = text.replace(':','')
        text = text.replace(',','')
        text = text.replace('!','')
        text = text.replace('?','')
        text=text.replace('\n','')
        text=text.replace('\t','')
        ham.append(text)

    except:
        pass
    

for text in spam_emails:
    text=email_to_plain(text)
    try:
        text = text.replace('.','')
        text = text.replace(':','')
        text = text.replace(',','')
        text = text.replace('!','')
        text = text.replace('?','')
        text = text.replace('\n','')
        text=text.replace('\t','')
        spam.append(text)

    except:
        pass
label=[0]*len(ham)+[1]*len(spam)
# Extract text
texts = ham+spam 
# Extract target
target = label 

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
wl=50000
tokenizer=Tokenizer(wl,filters='!@#$%^&*{}]~`[',lower=True)
tokenizer.fit_on_texts(texts)
text=tokenizer.texts_to_sequences(texts)
msl=250
text=pad_sequences(text,msl)
from keras.layers import Embedding, LSTM,Dense,Dropout,Dense
from keras.models import Sequential

Using TensorFlow backend.


In [9]:
from sklearn.model_selection import train_test_split

from keras.utils import to_categorical
y=to_categorical(target)
x_train,x_test,y_train,y_test=train_test_split(text,y,test_size=0.3, random_state=40)

In [10]:
model=Sequential()
model.add(Embedding(50000,64,input_length=250))
model.add(LSTM(64,input_shape=(250,10),activation='tanh'))
model.add(Dense(10,activation='relu'))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [11]:
history=model.fit(x_train,y_train,epochs=10, validation_split=0.2 ,batch_size=10)

C:\Users\parth\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1708 samples, validate on 427 samples
Epoch 1/10
1708/1708 [==============================] - 35s 20ms/step - loss: 0.2502 - accuracy: 0.9122 - val_loss: 0.1167 - val_accuracy: 0.9578
Epoch 2/10
1708/1708 [==============================] - 34s 20ms/step - loss: 0.0215 - accuracy: 0.9947 - val_loss: 0.0604 - val_accuracy: 0.9836
Epoch 3/10
1708/1708 [==============================] - 33s 19ms/step - loss: 0.0075 - accuracy: 0.9977 - val_loss: 0.0947 - val_accuracy: 0.9766
Epoch 4/10
1708/1708 [==============================] - 32s 19ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0907 - val_accuracy: 0.9742
Epoch 5/10
1708/1708 [==============================] - 33s 19ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0792 - val_accuracy: 0.9813
Epoch 6/10
1708/1708 [==============================] - 34s 20ms/step - loss: 2.8972e-04 - accuracy: 1.0000 - val_loss: 0.0770 - val_accuracy: 0.9836
Epoch 7/10
1708/1708 [==============================] - 33s 19ms/step - l

In [14]:
accr = model.evaluate(x_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

916/916 [==============================] - 2s 2ms/step
Test set
  Loss: 0.038
  Accuracy: 0.992


In [18]:
y_pred=model.predict(x_test)
y_test
labels=[1,0]
y_test_label=[]
y_pred_label=[]

for i in y_test:
    y_test_label.append(labels[np.argmax(i)])
for i in y_pred:
    y_pred_label.append(labels[np.argmax(i)])

In [21]:
new_complaint = ['sdhfsdfhykldjfkl  h kkjhfsd jkhadk hudfhkjdfhj,h h ']

seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=250)
pred = model.predict(padded)
labels = ['ham','spam']
print( labels[np.argmax(pred)])

spam


In [22]:
new_complaint = ['todays weather is cool ,isnt it']

seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=250)
pred = model.predict(padded)
labels = ['ham','spam']
print( labels[np.argmax(pred)])

ham
